# Lepton weights with coffea.analysis_tools

In [1]:
import awkward as ak
import numpy as np
from coffea.analysis_tools import Weights

## Electron pt toy array
Let's consider a toy example where we have 4 events with a variable number of electrons each.

In [2]:
electron_pt = ak.Array([[10, 45],[25, 9, 50],[18], [24, 7]])
electron_pt

<Array [[10, 45], [25, 9, ... [18], [24, 7]] type='4 * var * int64'>

## Flatten the electron array
We want to define an array where the weight corresponding to each electron is stored. In order to do that, we have to flatten the electron array to get an array with the proper size.
One can always recovering the original array by unflattening.

In [3]:
electron_pt_flat, electron_counts = ak.flatten(electron_pt), ak.num(electron_pt)
electron_pt_flat

<Array [10, 45, 25, 9, 50, 18, 24, 7] type='8 * int64'>

In [4]:
ak.unflatten(electron_pt_flat, electron_counts)

<Array [[10, 45], [25, 9, ... [18], [24, 7]] type='4 * var * int64'>

We define `weights` in such a way that its length corresponds to the number of electrons in all the events.

In [5]:
nelectrons = ak.size(electron_pt_flat)
weights = Weights(nelectrons)
weights.weight()

array([1., 1., 1., 1., 1., 1., 1., 1.])

## Electron weight toy array
We define toy weights for the electrons, keeping the flattened shape.

In [6]:
w = 0.5*(electron_pt_flat > 15) + 1.1*(electron_pt_flat < 15)
w.tolist()

[1.1, 0.5, 0.5, 1.1, 0.5, 0.5, 0.5, 1.1]

In [7]:
weights.add('custom_weight', w)

## Compute per-electron weights
After adding the weigths, we can compute the total weight and get the flattened weights. In order to get the per-event structure, we can unflatten the `weights` array.

In [8]:
weights.weight()

array([1.1, 0.5, 0.5, 1.1, 0.5, 0.5, 0.5, 1.1])

In [9]:
ak.unflatten(weights.weight(), electron_counts)

<Array [[1.1, 0.5], [0.5, ... 0.5], [0.5, 1.1]] type='4 * var * float64'>

## Compute per-event weights
In order to get a per-event weight, we can multiply the electron weights in each event. Each entry of the product array corresponds now to the per-event scale factor to apply to each event.

In [10]:
ak.prod(ak.unflatten(weights.weight(), electron_counts), axis=1)

<Array [0.55, 0.275, 0.5, 0.55] type='4 * float64'>